O presente exercício é uma reprodução de uma tarefa do curso de ciência de dados da univesp

# COM530 - Visão Computacional

## Semana 5 - Segmentação
## Videoaula 15 - Prática de Segmentação Semântica com CNN

- Importando os pacotes e imagens
- Analisando as imagens
- Preparando os dados
- Construindo uma rede convolucional Unet
- Treinando a rede
- Avaliando os resultados

Observações: 
- Ativar a GPU (Menu [Runtime], opção [Change runtime type])
- Sem GPU, a execução deste caderno pode levar mais de 40 minutos.


#Importar os pacotes e carregar os dados.

O conjunto de dados utilizado nesta aula foi construído a partir de imagens de Microscopia Eletrônica, as quais estão disponíveis em: https://www.epfl.ch/labs/cvlab/data/data-em/

Essas imagens estão em alta-resolução e foram segmentadas em mitocôndrias e sinapses.

A partir dessas imagens, foram geradas "sub-imagens" 256x256 a partir das regiões das imagens originais. Essas imagens 256x256 são as utilizadas nesta aula.

In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.utils import normalize
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
import glob

#Carregando as imagens

O conjunto de dados utilizado nesta aula foi construído a partir de imagens de Microscopia Eletrônica, as quais estão disponíveis em: https://www.epfl.ch/labs/cvlab/data/data-em/

Essas imagens estão em alta-resolução e foram segmentadas em mitocôndrias e sinapses.

A partir dessas imagens, foram geradas "sub-imagens" 256x256 a partir das regiões das imagens originais. Essas imagens 256x256 são as utilizadas nesta aula.

In [ ]:
!wget https://www.dropbox.com/s/rntsian2o49lzc5/256.zip?dl=1 -O 256.zip
!mkdir -p tmp/mitochondria/

local_zip = '256.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/mitochondria/')
zip_ref.close()

In [ ]:
#!wget https://www.epfl.ch/labs/cvlab/data/data-em/

!wget https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/volumedata.tif